Passo 1 - Instalar todas as depedências necessárias:

In [ ]:
!pip install pydub
!pip install nltk
!pip install ffmpeg-python
!apt-get update && apt-get install -y ffmpeg
!pip install git+https://github.com/openai/whisper.git

Passo 2 - Importar bibliotecas e preparar o ambiente


In [ ]:
from google.colab import files
import os
import nltk
from pydub import AudioSegment
from IPython.display import Audio, display

nltk.download('punkt')

print("✅ Todas as bibliotecas foram carregadas com sucesso!")

Passo 3 - Upload do arquivo e funções para processamento

In [ ]:
from google.colab import files
uploaded = files.upload()  # Aqui guardar o resultado em uma variável
file_name = list(uploaded.keys())[0]
print(f"Arquivo recebido: {file_name}")

In [ ]:
def convert_audio(input_file):
    print(f"Convertendo {input_file} para WAV...")
    audio = AudioSegment.from_file(input_file)
    wav_file = input_file.split('.')[0] + '.wav'
    audio.export(wav_file, format="wav")
    print(f"✅ Arquivo convertido: {wav_file}")
    return wav_file

wav_file = convert_audio(file_name)

# Mostrar áudio
print("\nPré-visualização do áudio:")
from IPython.display import Audio, display
display(Audio(wav_file))

Passo 4 - Transcrever o arquivo

In [ ]:
# Função de transcrição
def transcribe_audio(audio_file):
    print("Carregando modelo Whisper (small)...")
    import whisper
    model = whisper.load_model("small")

    print(f"Transcrevendo o áudio... (isso pode demorar alguns minutos)")
    result = model.transcribe(audio_file)

    return result["text"]

# Agora use a função
print("Iniciando transcrição...")
transcribed_text = transcribe_audio(wav_file)

# Exibindo parte do texto transcrito (para verificar)
print("Prévia da transcrição:")
print(transcribed_text[:300] + "..." if len(transcribed_text) > 300 else transcribed_text)

Passo 5 - Formata o texto da transcrição

In [ ]:
def format_text_simple(text):
    # Limpar espaços extras
    text = ' '.join(text.split())

    # Dividir em frases usando pontuação (método simples)
    sentences = []
    current = ""

    for char in text:
        current += char
        if char in ['.', '!', '?'] and len(current.strip()) > 0:
            sentences.append(current.strip())
            current = ""

    if current.strip():
        sentences.append(current.strip())

    # Formatação das frases
    formatted_text = ""
    for i, sentence in enumerate(sentences):
        # Capitalizar a primeira letra
        if len(sentence) > 0:
            sentence = sentence[0].upper() + sentence[1:]

        formatted_text += sentence + " "

        # Quebra de parágrafo a cada 4 frases
        if (i + 1) % 4 == 0:
            formatted_text += "\n\n"

    return formatted_text

# Usa a função "simples"
formatted_text = format_text_simple(transcribed_text)

# Exibe o resultado formatado
print("\n=== TEXTO FORMATADO ===")
print(formatted_text[:500] + "..." if len(formatted_text) > 500 else formatted_text)

# Salva em arquivo e fazer download
output_file = "transcricao.txt"
with open(output_file, 'w', encoding='utf-8') as f:
    f.write(formatted_text)

# Download do arquivo final
from google.colab import files
files.download(output_file)
print(f"✅ Concluído! Texto salvo em: {output_file}")